In [5]:
# --- 0. 安装必要的库 ---
# LDA需要 'gensim'
# 交互式可视化需要 'pyLDAvis'
!pip install -q gensim pyLDAvis

# --- 1. 导入库 ---
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm # (pyLDAvis不一定使用,但保留)
import os
import sys
import re

# 导入 gensim
try:
    from gensim.corpora import Dictionary
    from gensim.models import LdaModel
except ImportError:
    print("错误：无法导入 gensim。")
    print("请确保Kaggle Notebook已开启网络，并已成功运行 !pip install gensim。")
    sys.exit()

# 导入 pyLDAvis
try:
    import pyLDAvis
    import pyLDAvis.gensim_models as gensim_vis
    # 在Kaggle/Jupyter中启用可视化
    pyLDAvis.enable_notebook()
except ImportError:
    print("错误：无法导入 pyLDAvis。")
    print("请确保Kaggle Notebook已开启网络，并已成功运行 !pip install pyLDAvis。")
    sys.exit()

# --- 2. 设置字体和文件路径 (Kaggle路径) ---
# 尽管pyLDAvis是网页组件,我们仍然设置字体以防万一
input_dir = '/kaggle/input/nssfc-2025/'

font_path_chinese = os.path.join(input_dir, 'simkai.ttf')
font_path_english = os.path.join(input_dir, 'times.ttf')
data_file = os.path.join(input_dir, 'nssfc-fenci.csv')

# 检查字体文件
if os.path.exists(font_path_chinese):
    fm.fontManager.addfont(font_path_chinese)
if os.path.exists(font_path_english):
    fm.fontManager.addfont(font_path_english)

# --- 3. 加载和准备数据 ---
print(f"正在从 '{data_file}' 加载数据...")
try:
    df = pd.read_csv(data_file)
except FileNotFoundError:
    print(f"错误: 数据文件 '{data_file}' 未找到。")
    print("请确保您的数据文件和字体文件已上传到Kaggle的 '/kaggle/input/nssfc-2025/' 数据集中。")
    sys.exit()

print("数据加载成功。")

正在从 '/kaggle/input/nssfc-2025/nssfc-fenci.csv' 加载数据...
数据加载成功。


In [6]:
# 筛选 '管理学'
target_discipline = '管理学'
management_df = df[df['涉及学科'] == target_discipline].copy()

if management_df.empty:
    print(f"错误：未能在数据中找到学科门类 '{target_discipline}'。")
    sys.exit()

# 准备文本数据：'{项目名称分词}' 是一个字符串，需要按空格分割
print("正在准备 '管理学' 文本数据...")
# 确保数据是字符串类型并分割
texts = management_df['课题名称分词'].dropna().apply(lambda x: str(x).split()).tolist()

if not texts:
    print("错误：'管理学' 分词数据为空。")
    sys.exit()
    
print(f"共找到 {len(texts)} 篇 '管理学' 文档。")

# --- 4. 文本预处理 (新增：移除停用词) ---
print("正在准备 '管理学' 文本数据...")
# 1. 加载原始分词文本
texts_raw = management_df['课题名称分词'].dropna().apply(lambda x: str(x).split()).tolist()

# 2. 定义常见的项目名称停用词 (您可以根据需要增删)
# 1. 研究元词汇与方法论相关
research_meta_stopwords = [
    '影响', '体系', '视域', '机理', '效应', '构建', '建构', '理论', '实践', 
    '对策', '视角', '背景', '基于', '模式', '应用', '问题', '评估', '测度', 
    '关系', '形成', '重构', '逻辑', '形态', '系统', '模型', '演化', '演变', 
    '经验', '资料', '史料', '要素', '研究', '机制', '路径', '及其'
]

# 2. 高度泛化的背景词
generalized_context_stopwords = [
    '中国', '社会', '我国', '国家', '时代', '当代', '时期', '历史', 
    '世纪', '1949', '近代'
]

# 3. 抽象的过程、目标与修饰词
abstract_process_stopwords = [
    '策略', '提升', '优化', '整理', '驱动', '赋能', '建设', '实现', '应对', 
    '促进', '困境', '动态', '识别', '效能', '交融'
]

# 合并所有停用词
my_stop_words  = research_meta_stopwords+generalized_context_stopwords+abstract_process_stopwords
print(f"将从文本中移除 {len(my_stop_words)} 个自定义停用词...")

# 3. 从 texts 中移除停用词
texts_processed = [
    [word for word in doc if word not in my_stop_words]
    for doc in texts_raw
]

if not texts_processed:
    print("错误：移除停用词后所有文档均为空。")
    sys.exit()

print("停用词移除完成。")

# --- 创建 Gensim 词典和语料库 (使用处理后的文本) ---
print("正在创建 Gensim 词典和语料库...")
# 使用处理过的文本 (texts_processed) 创建词典
id2word = Dictionary(texts_processed)

# 过滤词典：
# no_below=2: 移除那些在总共不到2个文档中出现过的词
# no_above=0.5: 移除那些在超过50%的文档中都出现过的词 (高频词)
id2word.filter_extremes(no_below=2, no_above=0.5)

# 使用处理过的文本和过滤后的词典创建语料库 (BoW)
corpus = [id2word.doc2bow(text) for text in texts_processed]

# 检查语料库是否为空
if not any(corpus):
    print("错误：语料库为空，可能是过滤条件过于严格。")
    sys.exit()

print(f"词典和语料库创建完成。词典大小 (过滤后): {len(id2word)}")

正在准备 '管理学' 文本数据...
共找到 590 篇 '管理学' 文档。
正在准备 '管理学' 文本数据...
将从文本中移除 61 个自定义停用词...
停用词移除完成。
正在创建 Gensim 词典和语料库...
词典和语料库创建完成。词典大小 (过滤后): 543


In [7]:
# --- 6. 训练最终的 LDA 模型 (K=14) ---
# --- 微调：主题数设置为 14 ---
num_topics = 14
print(f"正在训练最终的 LDA 模型, 主题数 K={num_topics}...")

lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    random_state=2025,
    passes=15, # 使用更多遍数进行最终训练
    chunksize=100
)
print("LDA 模型训练完成。")

# --- 7. 准备并保存 pyLDAvis 交互式可视化 ---
print("正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...")

# 这是将 gensim 模型转换为 pyLDAvis 所需数据的核心步骤
vis = gensim_vis.prepare(lda_model, corpus, id2word)

# 保存为 HTML 文件
output_filename = 'management_lda_k14_interactive.html'
try:
    pyLDAvis.save_html(vis, output_filename)
    print(f"交互式图表已成功保存为: {output_filename} (位于 /kaggle/working/ 目录)")
except Exception as e:
    print(f"保存交互式HTML时出错: {e}")

# --- 8. 在Kaggle Notebook中显示交互式图表 ---
print("\n" + "="*50)
print("正在Kaggle Notebook中显示交互式图表...")
print("="*50)

# 在Kaggle单元格的最后一行返回 'vis' 对象，它会自动渲染
vis

# --- 9. 打印每个主题下概率最高的10个主题词 ---
print("\n" + "="*50)
print(f"主题词分布 (Top 10 per topic) for '{target_discipline}'")
print("="*50)

for topic_id in range(num_topics):
    print(f"\n【主题 {topic_id}】")
    # 获取该主题下权重最高的10个词及其概率
    top_terms = lda_model.show_topic(topic_id, topn=10)
    for rank, (term, prob) in enumerate(top_terms, start=1):
        print(f"{rank:>2}. {term:<15s} 概率: {prob:.4f}")


正在训练最终的 LDA 模型, 主题数 K=14...
LDA 模型训练完成。
正在准备 pyLDAvis 交互式可视化 (这可能需要几秒钟)...
交互式图表已成功保存为: management_lda_k14_interactive.html (位于 /kaggle/working/ 目录)

正在Kaggle Notebook中显示交互式图表...

主题词分布 (Top 10 per topic) for '管理学'

【主题 0】
 1. 安全              概率: 0.0880
 2. 行为              概率: 0.0474
 3. 风险              概率: 0.0456
 4. 粮食              概率: 0.0401
 5. 韧性              概率: 0.0359
 6. 治理              概率: 0.0324
 7. 协同              概率: 0.0318
 8. 能源              概率: 0.0282
 9. 政策              概率: 0.0265
10. 链主              概率: 0.0260

【主题 1】
 1. 错配              概率: 0.0018
 2. 电池              概率: 0.0018
 3. 敏捷              概率: 0.0018
 4. 框架              概率: 0.0018
 5. 结构性             概率: 0.0018
 6. 隐私              概率: 0.0018
 7. 工具              概率: 0.0018
 8. 进口              概率: 0.0018
 9. 成长              概率: 0.0018
10. 产业化             概率: 0.0018

【主题 2】
 1. 数字              概率: 0.1569
 2. 电商              概率: 0.0537
 3. 生成              概率: 0.0448
 4. 市场              概率: 0.0391
 5. 政策           

In [8]:
# 在Kaggle单元格的最后一行返回 'vis' 对象，它会自动渲染
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.167003  0.057968       1        1  10.266326
12    -0.085364 -0.097451       2        1   9.879261
7      0.135679  0.297455       3        1   9.825842
11    -0.161733 -0.026375       4        1   9.261798
2      0.062754  0.003836       5        1   8.536354
6     -0.032901  0.210978       6        1   8.519315
3     -0.006566 -0.123692       7        1   7.998684
4      0.039870 -0.040392       8        1   7.112751
9     -0.096546 -0.094594       9        1   6.908962
13    -0.018249  0.009181      10        1   6.092176
5     -0.133161  0.020750      11        1   6.054722
10     0.219009 -0.097670      12        1   4.525165
8      0.202886 -0.096657      13        1   4.059894
1      0.041325 -0.023337      14        1   0.958751, topic_info=   Term       Freq      Total Category  logprob  loglift
53   创新  68.000000  68.000000  Default  30.0000  30.0000
13   智能  75.000000  75.000000  Default  29.0000  29.0000
11   人工  67.000000  67.000000  Default  28.0000  28.0000
69   数字  51.000000  51.000000  Default  27.0000  27.0000
39   发展  33.000000  33.000000  Default  26.0000  26.0000
..  ...        ...        ...      ...      ...      ...
12   政府   0.066618   8.198454  Topic14  -6.2971  -0.1654
13   智能   0.066618  75.142303  Topic14  -6.2971  -2.3809
14   监管   0.066618   7.280331  Topic14  -6.2971  -0.0467
15   保障   0.066618   9.898654  Topic14  -6.2971  -0.3539
16  劳动者   0.066618   5.817299  Topic14  -6.2971   0.1777

[551 rows x 6 columns], token_table=      Topic      Freq Term
term                      
424       3  0.822568  一体化
502       2  0.823167   上市
486       6  0.904893   下数
161      13  0.628234  下西部
71       11  0.883483   专精
...     ...       ...  ...
512       9  0.882629   食品
238       6  0.570677   高原
235       2  0.762553  高标准
164       4  0.635815   高碳
479      12  0.885919   高端

[466 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 13, 8, 12, 3, 7, 4, 5, 10, 14, 6, 11, 9, 2])